In [1]:
from copy import deepcopy
import os

import fastplotlib as fpl
import numpy as np

import mesmerize_core as mc
import mesmerize_viz

In [2]:
imgs = [np.random.rand(100, 100, 100)] * 3
iw = fpl.ImageWidget(imgs)
iw.show()

RFBOutputContext()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [3]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

In [4]:
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

In [5]:
# Mac users!
# temporary patch for Mac, won't be necessary in next release
# Thanks Ryan Ly for the PR! :D I need to dig into it more before merging
# conda_prefix_1_str = os.environ['CONDA_PREFIX'].replace(os.path.join(' ', 'envs', 'mescore')[1:], '')
# os.environ['CONDA_PREFIX_1'] = conda_prefix_1_str

# Set `caiman_data` path

In [6]:
mc.set_parent_raw_data_path("/Users/js0403/caiman_data/")

PosixPath('/Users/js0403/caiman_data')

In [7]:
batch_path = mc.get_parent_raw_data_path().joinpath("minimal_test/batch.pickle")

In [8]:
# create a new batch, delete if exist (for test purpose only, not for your real batches!)
df = mc.create_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


In [9]:
mcorr_params = {
    "main":{
        'max_shifts': [12, 12],
        'strides': [32, 32],
        'overlaps': [20, 20],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None,
    }
}

In [10]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=mc.get_parent_raw_data_path().joinpath("example_movies/demoMovie.tif"),
    params=mcorr_params,
    item_name="demo",
)

In [11]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,demo,example_movies/demoMovie.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",None,2023-11-08T12:27:27,None,None,None,dfc262c9-68e9-4dc4-bc30-dbc82bdba8f9


In [11]:
df.iloc[-1].caiman.run()

df = df.caiman.reload_from_disk()

starting mc
mc finished successfully!
computing projections
Computing correlation image


100%|██████████| 1/1 [00:00<00:00, 45.59it/s]


finished computing correlation image


In [12]:
viz = df.mcorr.viz(data_options=["input", "mcorr"], image_widget_kwargs={"grid_plot_kwargs": {"size": (700, 300)}})
viz.show()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [13]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.9,
            'rf': 20,
            'stride': 12, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

In [14]:
df.caiman.add_item(
    algo="cnmf",
    input_movie_path=df.iloc[0],
    params=params_cnmf,
    item_name="demo",
)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,demo,example_movies/demoMovie.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",{'mean-projection-path': 19ddaf1f-5520-4dfb-a6...,2023-11-09T17:12:56,2023-11-09T17:13:16,10.5 sec,None,19ddaf1f-5520-4dfb-a61d-391ba6b6007f
1,cnmf,demo,19ddaf1f-5520-4dfb-a61d-391ba6b6007f/19ddaf1f-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",None,2023-11-09T17:13:21,None,None,None,d12c4836-7ac3-4a01-9cd9-b4ed26ace4d7


In [17]:
df.iloc[-1].caiman.run()
df = df.caiman.reload_from_disk()

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                                        demo
input_movie_path    19ddaf1f-5520-4dfb-a61d-391ba6b6007f/19ddaf1f-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2023-11-09T17:13:21
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             d12c4836-7ac3-4a01-9cd9-b4ed26ace4d7
Name: 1, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.9, 'rf': 20, 'stride': 12, 'K': 4, 'gSig': (4, 4), 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_c

2023-11-09 17:14:10.118094: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 124ms/step


In [18]:
viz = df.cnmf.viz()
viz.show()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

RFBOutputContext()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


RFBOutputContext()

/Users/js0403/anaconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")
